# Use Case 5: Gene set enrichment analysis

<b>Import standard data analysis imports, as well as the gseapy which will allow us to perform a Gene set enrichment analysis</b>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import gseapy as gp

<b>Import the CPTAC data</b>

In [3]:
import CPTAC

Loading Clinical Data...
Loading Proteomics Data...
Loading Transcriptomics Data...
Loading CNA Data...
Loading Phosphoproteomics Data...
Loading Somatic Data...

 ******PLEASE READ******


<b>Retrieve the clinical and proteomics dataframes</b>

In [4]:
clinical = CPTAC.get_clinical()
proteomics = CPTAC.get_proteomics()

<b>For this example we will be separating the protein abudance based on the clinical MSI. Our first step is to combine the MSI information into the proteomics dataframe utilizing the <code>CPTAC.compare_clinical()</code> function</b>

In [5]:
msiProt = CPTAC.compare_clinical(clinical, proteomics, 'MSI')

<b>Separate the proteomics into two groups based on whether MSI is MSI-H or other </b>

In [6]:
high = msiProt['MSI'] == "MSI-H"
other = msiProt['MSI'] != "MSI-H"
highMSI = msiProt[high]
otherMSI = msiProt[other]

<b>Find which genes are upregulated in each partition</b>

In [7]:
genes_passing = []
genes = highMSI.columns[1:]
threshold = .05 / len(genes)
for gene in genes:
    high_abundance = highMSI[gene]
    other_abundance = otherMSI[gene]
    pvalue = stats.ttest_ind(high_abundance, other_abundance, equal_var = False, nan_policy='omit').pvalue
    if pvalue < threshold:
        genes_passing.append(gene)
print(genes_passing)

['AAK1', 'AASS', 'ABI2', 'ABI3BP', 'ABL1', 'ACD', 'ACLY', 'ACTN1', 'ACTN4', 'ADARB1', 'ADIRF', 'ADO', 'AEBP1', 'AFAP1L2', 'AGAP3', 'AGFG2', 'AGO1', 'AGO2', 'AGO3', 'AGR2', 'AHDC1', 'AHNAK', 'AKAP12', 'AKT3', 'ALDH18A1', 'ALDH1A2', 'ALKBH4', 'ANG', 'ANGPTL2', 'ANK2', 'ANKZF1', 'AOC3', 'AP1G2', 'AP1M2', 'APCS', 'ARCN1', 'ARFIP1', 'ARFIP2', 'ARHGAP23', 'ARHGAP31', 'ARHGAP6', 'ARHGEF16', 'ARHGEF17', 'ARMCX1', 'ARPC1A', 'ASPN', 'AVL9', 'BAG3', 'BAIAP2L1', 'BCL7C', 'BGN', 'BSPRY', 'C11orf68', 'C11orf96', 'C7', 'CA3', 'CAB39L', 'CACNA2D1', 'CACNB3', 'CACYBP', 'CALCRL', 'CALD1', 'CAMK2G', 'CAMSAP3', 'CAND2', 'CAP2', 'CAPN2', 'CASKIN2', 'CASP3', 'CASP6', 'CAV1', 'CAVIN1', 'CAVIN2', 'CAVIN3', 'CBFB', 'CBL', 'CBX1', 'CCDC102A', 'CCDC134', 'CCDC50', 'CCDC91', 'CCNA2', 'CD109', 'CD2AP', 'CD34', 'CD93', 'CDC42BPB', 'CDC42EP2', 'CDC42EP3', 'CDC42EP4', 'CDH13', 'CENPV', 'CEP112', 'CEP68', 'CFB', 'CFHR2', 'CFL2', 'CGN', 'CHAF1A', 'CHAF1B', 'CHL1', 'CIB1', 'CIC', 'CLIC4', 'CLPX', 'CNN1', 'CNRIP1', 'COG2

In [7]:
#TODO: how to pass otherMSI column into apply function? otherMSI[x] doesn't work. Should I combine the dataframes?
#passing2 = highMSI[highMSI.columns[1:]].apply(lambda x:stats.ttest_ind(x, otherMSI[x])) 
#passing2

<b>Then use the genes that are up-regulated in these partitions to do a gene set enrichment analysis</b>

In [8]:
enr = gp.enrichr(gene_list = genes_passing, description='MSI partitions', gene_sets='KEGG_2016', outdir='test/enrichr_kegg',cutoff=.5)
enr.res2d

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Z-score,Combined Score,Genes,Gene_set
0,Focal adhesion_Homo sapiens_hsa04510,39/202,7.398323e-21,1.635029e-18,8.693608e-17,1.921287e-14,-1.922008,89.090885,TNXB;ROCK1;ROCK2;LAMA4;ILK;MYLK;THBS3;AKT3;CAP...,KEGG_2016
1,Proteoglycans in cancer_Homo sapiens_hsa05205,29/203,2.486726e-12,2.747832e-10,7.098578e-10,7.843928e-08,-1.987454,53.104871,ROCK1;ROCK2;HSPB2;ITPR1;CBL;FGF2;RRAS;CASP3;AK...,KEGG_2016
2,Regulation of actin cytoskeleton_Homo sapiens_...,27/214,2.584886e-10,1.904199e-08,3.135844e-08,2.107788e-06,-1.871449,41.314424,ROCK1;ROCK2;ARPC1A;FGF2;MYLK;RRAS;CFL2;GNA12;P...,KEGG_2016
3,Vascular smooth muscle contraction_Homo sapien...,20/120,3.956416e-10,2.185920e-08,3.815002e-08,2.107788e-06,-1.789354,38.740440,GUCY1A2;PPP1R14A;CALCRL;PPP1R12A;ROCK1;ROCK2;I...,KEGG_2016
4,Bacterial invasion of epithelial cells_Homo sa...,14/78,6.480486e-08,2.864375e-06,2.078733e-06,9.188002e-05,-1.667587,27.601713,SEPT11;CAV1;ARPC1A;GAB1;ILK;CBL;SEPT2;PTK2;SEP...,KEGG_2016
5,Oxytocin signaling pathway_Homo sapiens_hsa04921,18/158,1.159587e-06,3.660983e-05,2.466891e-05,7.710170e-04,-1.814166,24.795014,GUCY1A2;PPP1R12A;ROCK1;ROCK2;CACNA2D1;ITPR1;PR...,KEGG_2016
6,Tight junction_Homo sapiens_hsa04530,18/139,1.689420e-07,6.222695e-06,5.007797e-06,1.844539e-04,-1.578107,24.608541,ACTN1;PRKCA;ACTN4;CGN;TJP1;OCLN;RRAS;PPP2R1B;A...,KEGG_2016
7,Pathways in cancer_Homo sapiens_hsa05200,29/397,9.035108e-06,1.996759e-04,2.181325e-04,3.708253e-03,-1.869539,21.713554,ROCK1;ROCK2;LAMA4;CBL;FGF2;GNG7;SUFU;CASP3;AKT...,KEGG_2016
8,Arrhythmogenic right ventricular cardiomyopath...,12/74,1.754641e-06,4.847195e-05,2.791012e-05,7.710170e-04,-1.578629,20.921955,DSP;CACNB3;DES;SGCD;JUP;SGCB;CACNA2D1;ACTN1;LM...,KEGG_2016
9,Hypertrophic cardiomyopathy (HCM)_Homo sapiens...,12/83,6.071468e-06,1.490883e-04,7.620987e-05,1.871376e-03,-1.588528,19.081253,CACNB3;DES;SGCD;TPM4;SGCB;TPM2;CACNA2D1;MYL3;L...,KEGG_2016
